<img src="https://justaklikaway.files.wordpress.com/2014/05/shut-up-and-take-my-money.jpg" height="400" width="400"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 1: продажи </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Первой такой аркой будут продажи. Им будет посвящена первая серия из семинаров и домашек. 


#  Эпизод III (месть менеджера) 

Ну вот и всё, модель готова. Она прогнозирует продажи. Когда нас попросили сделать её, предполагалось, что она будет решать какую-то реальную сложную проблему и сэкономит нам гору бабла. Давайте посмотрим насколько успешно она с этим справляется.

In [1]:
# Как обычно, для начала подгружаем нужные нам пакеты
import pandas as pd    # пакет для работы с таблицами 
import numpy as np     # пакет для работы с матрицами 

# пакеты для картиночек 
import matplotlib

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')  # правильный (наиболее красивый) стиль у графиков

%matplotlib inline

# 1. Тестовый период и модели

Подгрузим данные, которые мы отложили для тестового периода. Их наша модель ещё не видела. Интересно, как именно она на них себя покажет.

In [2]:
df = pd.read_csv('../sem1_data_prepare/sales_test_v1.tsv', sep='\t')    # подгружаем датасет
print('Размерность таблицы:', df.shape)      # shape показывает его размер
df.head()                    

Размерность таблицы: (73684, 148)


,Weekly_Sales,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Black_Friday,Pre_christmas
0,17147.44,151315,24.194444,3.749,21290.13,0.0,69.89,4977.35,3261.04,221.671800,...,0,0,0,1,0,0,0,0,0,0
1,18164.20,151315,23.205556,3.688,8351.40,0.0,10.52,2443.14,3127.88,221.725663,...,0,0,0,1,0,0,0,0,0,0
2,18517.79,151315,21.294444,3.630,6154.14,0.0,45.11,1675.49,5508.18,221.742674,...,0,0,0,1,0,0,0,0,0,0
3,16963.55,151315,25.122222,3.561,4039.39,0.0,745.19,1429.96,3631.13,221.744944,...,0,0,0,1,0,0,0,0,0,0
4,16065.49,151315,25.527778,3.501,6086.21,12.0,370.51,148.75,3690.85,221.747214,...,0,0,0,0,1,0,0,0,0,0


Подгружаем нашу модель. 

In [11]:
from sklearn.externals import joblib

model_lasso = joblib.load('../sem2_regression/model_lasso.pkl')
model_lasso

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [12]:
model_rf = joblib.load('../sem2_regression/model_rf.pkl')
model_rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Давайте построим небольшую табличку с прогнозами, с которой в дальнейшем и будем работать.

In [15]:
# это то, что мы пытаемся прогнозировать! 
y = df['ln_future_sales']

# то, по чему пытаемся прогнозировать, ln_Weekly_Sales оставили!
df = df.drop(['ln_future_sales', 'future_sales', 'Weekly_Sales'], axis=1)
X = df.get_values()

In [18]:
dfp = pd.DataFrame() 

dfp['real'] = y
dfp['lasso_predict'] = model_lasso.predict(X)
dfp['rf_predict'] = model_rf.predict(X)

In [19]:
dfp.head()

,real,lasso_predict,rf_predict
0,9.807263,9.752433,9.676369
1,9.826541,9.795832,9.846766
2,9.738881,9.824624,9.859840
3,9.684491,9.731945,9.737745
4,9.779454,9.688332,9.727878


# 2. Легенда

Всё, что описано в этом пункте - это легенда. Правду о своих магазинах знает только Walmart. 

* Будем считать, что мы работаем со скоропортящимся продуктом. Если люди в течение недели его не раскупают, он протухает. Приходится его выкидывать.
* Будем считать, что сейчас магазины работают очень-очень глупым образом: 

Все магазины делятся на три типа. В каждый мы специально закупаем слишком много продуктов. В магазины типа $A$ каждую неделю мы покупаем товара на $500000$, в магазины типа $B$ на $700000$, в магазины типа $C$ на $120000$. Всё, что остаётся в конце недели - списывается. Выходит, что те данные, которые мы видим в табличке - это реальный спрос на товар. 

Будем считать, что закупочные цены вдвоё ниже розничных. То есть, когда мы хотим продать товара на $500000$, как в случае магазина $A$, мы тритим на закупку $250000$. 

__Ситуация в будущем:__

Магазину надоело тратить деньги на закупку впустую. Возникает желание научиться оптимизировать закупки. Для этого есть две стратегии.

__Стратегия 1:__ Будем закупать на следущую неделю ровно столько продуктов, сколько было распродано за текущую неделю. Если потребители приходили в магазин и не могли купить товар, будем увеличивать закупку на $10000$.

__Стратегия 2:__ Обучим модель, которая будет прогнозировать продажи. Сколько она спрогнозировала, столько и будем привозить. 


### Задание 1 

__[1]__ Сколько денег мы потеряем на тестовом периоде из-за лишних закупок, если продолжим действовать по той же самой стратегии? 

__[3]__ Обучите линейную регрессию без подбора гиперпараметров. Выясните сколько денег она позволяет сэкономить на закупках в течение тестовго периода. Сколько при этом денег будет потеряно из-за перепрогноза? Сколько денег будет потеряно из-за недопрогноза? Правда ли, что линейная регрессия по ошибкам обоих типов оказывается лучше первой стратегии? 

__[3]__ Пусть ошибка недопрогноза для нас страшнее. Оцените насколько (это творчество). По аналогии [с примерами из документации](https://github.com/keras-team/keras/blob/master/keras/losses.py) реализуйте такую функцию потерь и обучите свою архитектуру, используя её. 

__[2]__ Последний пункт! Он состоит из балобалии. Предположим, что ваша итоговая архитектура - пушка! Она рвёт все бэйзлайны и вы собираетесь с довольным, улыбающимся менеджером катить в прод. Поначалу модель будет работать просто отлично. Однако через какое-то время начнут всплывать проблемы. Что это за проблемы? Какие соломинки можно было бы подстелить для их решения? 